# 🛰️ 🚀 An Introduction To Space Flight! 

🌌 Today, we will learn all about the basics of space flight! In lecture we talked a lot about what the 6 Keplerian elements mean, and how they map not only to an equivalent position and velocity, but they also describe trajectory of a spacecraft in two-body motion. In this IPyNB file, we will put into practice what we have learnt. Let's plot some orbits today!

**⚠️ Important Note #1: The very first thing you would have to run is the block of code below (press ▶️ above). This loads all the basic astrodynamics libraries written for this course, necessary for our lesson.** 

**⚠️ Important Note #2: You must have a folder 📁📂 named "spaceflight" in your current directory at the same level of your IPython notebook, with all Python helper files "orbits.py", "integrate.py", and "plots.py" in there.** 

In [ ]:
# Important! Run this block of code; don't edit it!
import sys
import piplite
await piplite.install('ipywidgets')
from ipywidgets import interactive, fixed
import numpy as np
sys.path.append("spaceflight")

import plots
import orbits
import integrate

# 🛰️ 🚀 Now we're ready to start propagating orbits! 

🌌 In the lecture, you learnt about describing orbits using the 6 orbital elements. Now let's try to plot out an example orbit using the 6 elements! The 6 Keplerian elements are beautiful because they can give us a complete description of the orbit analytically, without resorting to numerical approximations using integration. The alternative approach, using numerical integration, is also known as numerical propagation. Numerical propagation typically takes in an initial position and velocity, and simply performs integrations of the accelerations to get future velocities, and of the velocities to get future positions. This is possible because we know the gravitational force on a spacecraft thanks to Newton's Laws!

# 🛰️ 🚀 Run the code below for an interactive orbit!

**While you do so, think of what the pros and cons of analytical versus numerical orbit propagation are ❓❓❓**  

In [ ]:
# Run this block of code, but don't edit it!

# Orbital parameters:
# ---> a:  orbit semi-major axis [km]
# ---> e:  orbit eccentricity [unitless]
# ---> i:  orbit inclination [degrees]
# ---> w:  orbit argument of periapsis [degrees]
# ---> R:  orbit right ascension of the ascending node [degrees]
# ---> M:  orbit mean anomaly [degrees]

# Simulation related parameters:
# ---> t:         time duration of the propagation [s]
# ---> dt:        time step of the propagation [s]
# ---> azimuth:   viewing angle azimuth of the plot [degrees]
# ---> elevation: viewing angle elevation of the plot [degrees]

orbitWidget = interactive( plots.plot_orbit_interactive,
                           a=(6378.0,40000.0), e=(0.0,0.99), i=(0.0,180.0),
                           w=(-180.0,180.0), R=(-180.0,180.0), M=(0.0,360.0),
                           t=(0.0,86400), dt=(0.0,360.0),
                           azimuth=(0.0,360.0), elevation=(0.0,360.0))
orbitWidget

# You might get a warning once the plot updates too many times. Don't worry 
# about that, you can right-click anywhere in this cell and "Clear All Outputs".

🌌 If you don't see a full orbit being propagated, try increasing the total duration `t` of the simulation! If the orbit trajectory doesn't look smooth, perhaps you need a finer time step, try reducing `dt`. Toggle the viewing angles `azimuth` and `elevation` to visualize the plot from other angles. These two viewing angles do not affect the physics of the orbit simulation. Finally, play around with the orbit elements, `a, e, i, w, R, M` to see what plots you get!

# 🛰️ 🚀 Analytical Propagators

🌌 Next, let's take a look at analytical propagators in some detail. Using the Keplerian elements, we can describe the full trajectory, without the loss of accuracy accruing from numerical integration. From Kepler's Law of Harmonies, the rate of change of the mean anomaly is described by the mean motion $n$, which is a **constant of motion for a small satellite with negligible mass as compared to the primary body, in any conservative gravitational field** given by

$$n = \frac{dM}{dt} = \sqrt{\frac{GM}{a^3}}$$

We can use this fact, and the fact that the other 5 elements ( *a, e, i, $\omega$, $\Omega$* ) are invariant in any two-body system (thanks to the conservation of total mechanical energy), to propagate the orbit solely based on the mean anomaly.

In [ ]:
# Freely edit this block of code with your inputs!

# Orbital parameters:
# ---> a:  orbit semi-major axis [km]
# ---> e:  orbit eccentricity [unitless]
# ---> i:  orbit inclination [degrees]
# ---> w:  orbit argument of periapsis [degrees]
# ---> R:  orbit right ascension of the ascending node [degrees]
# ---> M:  orbit mean anomaly [degrees]

a = ...
e = ...
i = ...
w = ...
R = ...
M = ...

# Don't forget to declare the total simulation duration and the time step!
# The time step should be a relatively small value (< 5 minutes typically)

time_total = ... # Unit: seconds
time_step  = ... # Unit: seconds

# Don't forget also to hit the "play" button so that your variables are
# declared for the next simulation!

🌌 Recall that a set of 6 orbital elements map specifically to a unique set of 6 Cartesian coordinates. We can do this because the orbital elements specify the size (*a*), shape (*e*), and orientation (*i, $\omega$, $\Omega$*) of the orbit, as well as the unique angular position of the spacecraft along the orbit (the anomaly parameter). With this, we have enough information to satisfy the 6 degrees of freedom necessary to specific an object's position and velocity.

![Orbital Elements Visualization](https://kyleniemeyer.github.io/space-systems-notes/_images/orbital_elements.svg "Orbital Elements Visualization")

*Image credits: Kyle Niemeyer's GitHub*

🌌 **Try running the function below that converts the set of orbital elements you have defined above into a unique set of Cartesian coordinates (position XYZ and velocity XYZ)!**

In [ ]:
# Make sure you have ran the previous block of code before this one!
# Then, you may run this block of code, but don't edit it!

states = orbits.elements2states([a,e,i,w,R,M])
print("Cartesian states:",states, "[km]")

🌌 Since the mapping between cartesian inertial frame coordinates and the orbital elements should be unique, can we see if we can recover back our original orbital elements by converting: **elements ⏩ states ⏩ elements** ❓

In [ ]:
# You can use the function `orbits.states2elements( <list_of_states> )`
# where it takes in a list elements (thus a list of size 6)
print("Here are our original orbit elements!", [a,e,i,w,R,M], '\n')
print("Here are our states converted from elements!", states, '\n')

# TODO: Uncomment the line and edit your code below!
# elementsFromStates = orbits.states2elements( ... )

# No need to edit this line, it just prints the line you edited above.
print("Elements -> States -> back to Elements:", elementsFromStates, '\n') 

🌌 As the orbit propagates analytically via its mean motion, it generates a number N of discrete states, with this number N being the number of samples we want. The set of states is an 6xN matrix, where each of the 6 rows represent the positions X,Y,Z, and velocities X,Y,Z components.

🌌 **Let us do an analytical orbit propagation using the object's mean motion, and print out the first three states! Each state is being sampled at the time step you had specified above.**

In [ ]:
# Run this block of code, but don't edit it!
statesAnalytical = integrate.analytical(a, e, i, w, R, M, time_total, time_step)

# Print out the first three time stamps of coordinates!
print(statesAnalytical[:,0:3])

❗ **QUESTIONS!** ❗

- Do these coordinates "look intuitive" ❓
- What are the benefits of using the orbital elements, versus the typical position-velocity XYZ ❓
- Can you think of other coordinates that can describe the spacecraft position and/or velocity? ❓

🌌 Finally, let's plot the orbits once again, can you visually predict how your orbit would look like in terms of size, shape, and orientation, given your inputs? Feel free to play around with the parameters you defined above.

In [ ]:
plots.plot_orbit( statesAnalytical ) # Note, this will take awhile to load!

# 🛰️ 🚀 Numerical Propagators

In the analytical two-body world, propagating an orbit solely on the Keplerian elements uses the fact that the mean motion of a spacecraft is a **constant of motion in a conservative gravitational field**. Thus, the spacecraft follows the ellipse (with a periodic change in anomaly), while all other elements are constant.

In reality though, spacecraft are subject to forces beyond just the gravity of the Earth. First, the Earth itself is not perfectly spherical, and so even the gravitational forces of the Earth are very much dependent on the spacecraft's current latitude, longitude, and altitude. Second, other external bodies, such as the third body gravitational effects of the Sun ☀️ , the Moon 🌒, and other planets like Jupiter and Saturn 🪐 also exert their gravitational influences on their spacecraft, although very weakly. The Earth's atmosphere also exerts a tiny amount of drag in low Earth orbit, and the irradiance of the sun also exerts a radiation pressure on the spacecraft body. Taken altogether, while Earth's gravity is still dominant, over a long period of time, these perturbing forces will derail the prediction of our orbits using our analytical two-body orbit propagator.

More importantly, most times, we want control over our spacecraft dynamics! We have thrusters on board, we have propulsion, and we want to use our engines to reach for the stars! 🌠🌟

Our thrusters will most intuitively output a change of velocity best described as a vector $\Delta\vec{V}$ in XYZ coordinates. Most often, these XYZ coordinates aren't just defined in any inertial coordinate system, but they are defined in a reference frame "local" to the spacecraft, such as the local body frame, or the Hill frame.

![Orbit frames](https://i.stack.imgur.com/9NsPo.png "Orbit frames")

Thus, propagating our orbits numerically, allows us to incorporate more external forces into our force model, such as thrust or drag, instead of relying on a constant of motion such as the mean motion (in the analytical propagator) where we otherwise have no avenue to add these additional forces.

🌌 **Let us try propagating an orbit using both the analytical approach and using numerical integration of the dynamics! First, you will need to specify your orbit parameters, as you had done before.**

In [ ]:
# Freely edit this block of code with your inputs!

# Orbital parameters:
# ---> a:  orbit semi-major axis [km]
# ---> e:  orbit eccentricity [unitless]
# ---> i:  orbit inclination [degrees]
# ---> w:  orbit argument of periapsis [degrees]
# ---> R:  orbit right ascension of the ascending node [degrees]
# ---> M:  orbit mean anomaly [degrees]

a = ...
e = ...
i = ...
w = ...
R = ...
M = ...

# Don't forget to declare how long you would like to propagate, and the time step!
# The time step should be a relatively small value (less than 5 minutes typically)
time_total = ... # Unit: seconds
time_step  = ... # Unit: seconds

🌌 Now, let's propagate and plot the orbit using the **analytical two-body solution** which we had done before.

In [ ]:
# Run this block of code, but don't edit it!
statesAnalytical = integrate.analytical(a, e, i, w, R, M, time_total, time_step)
plots.plot_orbit(statesAnalytical)

🌌 Then, let's propagate and plot the orbit using **pure numerical integration**.

In [ ]:
# Run this block of code, but don't edit it!
states = orbits.elements2states([a,e,i,w,R,M])
statesNumerical = integrate.numerical(states, time_total, time_step)
plots.plot_orbit(statesNumerical)

🌌 **Question! What do you notice about both the orbits propagated by both techniques** ❓

# 🛰️ 🚀 Let's fire the thrusters ❕ ❤️‍🔥

🌌 In this exercise, we are going to investigate what happens to the shape of our orbits once we start firing those engines! Do you think they will still remain circular or elliptical? In this exercise, we are going to specify our velocity impulses in the three cardinal directions of the **Hill Frame**:

- **The Radial direction** $\hat{i}$: the unit direction vector pointing from the Earth's center to the spacecraft body
- **The Transverse direction** $\hat{j}$: the unit direction vector pointing along the velocity of the spacecraft.
- **The Normal direction** $\hat{k}$: the unit direction vector pointing normal to the plane of the orbit.

In other words, if we were to specify a velocity impulse of **one kilometer per second**, in each RTN direction that would be:

$$
\begin{aligned}
\Delta V_R & = [ \ 1, \ 0, \ 0 \ ]^T \\
\Delta V_T & = [ \ 0, \ 1, \ 0 \ ]^T \\
\Delta V_N & = [ \ 0, \ 0, \ 1 \ ]^T
\end{aligned}
$$

Note that in this entire tutorial, we have been sticking to kilometers for distance units, so watchout for unit errors!

In the block of code below, we will have a demonstration scenario on how you can work on injecting additional velocity into your spacecraft constantly throughout its flight.

🌌 **Demonstration code:** Let us see what happens if we were to propagate the satellite for one full day, and every 1 minute, we would inject an additional $\Delta V = 1.0$ m/s in the same direction as its current velocity (transverse direction). A helper function has been provided to convert your velocity impulse into the inertial frame. **You don't have to edit the first block of code since it exists purely for demonstration**.

In [ ]:
# Run this code, and study it, but you don't have to edit it unless you want!

# Again, let's come up with our set of orbital elements.

a = ... # Edit this! Units in km
e = ... # Edit this! Unit-less
i = ... # Edit this! Units in degrees
w = ... # Edit this! Units in degrees
R = ... # Edit this! Units in degrees
M = ... # Edit this! Units in degrees

# Now, let's convert our defined orbital elements into a Cartesian state, so
# that it is easier to work with! Recall that a state vector comprises of:
# [pos_x, pos_y, pos_z, vel_x, vel_y, vel_z].

# Question: Why work with Cartesian states instead of the orbital elements?

states = orbits.elements2states([a,e,i,w,R,M])

# Next, let's define our total simulation time `t`, time step `dt`, and the
# Delta-V impulse `dv` which will be applied at every step `dt`.

t = 86400
dt = 60

# Feel free to change this Delta-V value and watch how the shapes of orbits
# change with different directional maneuvers switched on!

dv = [ 0.0, 0.001, 0.0 ] # Default: a forward thrust +1 m/s per minute.

# Let's introduce an augmented numerical integration function:
# ---> numerical_continuous_dv( states, t, dt, dv )

statesIntegrated = integrate.numerical_continuous_dv( states, t, dt, dv )

# This time, instead of taking in just the initial state, the total time of 
# integration, and the time step, it will additionally also take in a velocity
# impulse as a 1x3 NumPy array. This velocity impulse will be applied at
# every time step (think of the thrusters operating in a "continuous" mode).

# Now we can plot this orbit!
plots.plot_orbit( statesIntegrated )

🌌 **What happened here?** Well, (assuming you didn't change the default $\Delta V$), we had switched on an engine that provides a $\Delta V$ of `dv = 1 m/s` injected in the spacecraft for every `dt = 60` seconds, running for one full day! Notice that this was applied in the transverse direction (in the same direction as the current velocity vector), and so as a result, the spacecraft's linear velocity speeds up - the total mechanical energy of the orbit increases, and thus the orbit gradually widens with time. In reality, switching on our thrusters all the time is not feasible due to spacecraft size-weight-and-power constrains. Typically, we can control the trajectory of an orbit using a well-timed, single thrust. We will do this in our final summary exercise.



# 🛰️ 🚀 **Challenge Exercise**: Above and Beyond ❕ ❤️‍🔥

🌌 Here it is, the summary exercise! We are going to compute what the thrust value should be in terms of $\Delta V$ (km/s) in order to perform the Hohmann transfer you just learnt about in class, from a circular orbit with a semi-major axis of 7,000 km to 10,000 km.

🌌 **Task: Compute the values of $\Delta V_1$, $\Delta V_2$, and the time of flight between them, to complete the Hohmann transfer.** 

🌌 *Hint: You may find the following equations we covered in lecture useful*

$$
\begin{aligned}
GM & = 398600.4418 \quad (km^3/s^2) \qquad & \text{(Earth gravity constant)} \\
T & = 2 \pi \sqrt{\frac{a^3}{GM}} \qquad & \text{(Period of One Orbit)} \\
V_{circle} & = \sqrt{ GM \left( \frac{1}{r} \right) } \qquad & \text{(Orbit velocity, circular)} \\
V_{ellipse} & = \sqrt{ GM \left( \frac{2}{r} - \frac{1}{a} \right) } \qquad & \text{(Orbit velocity, ellipse)}
\end{aligned}
$$

In [ ]:
# Final exercise! Input the two Delta-V values that accrue from your thrust,
# and also the wait time between each thrust maneuver. You can use this
# cell to do your computations, or you can do your computations by hand
# on a piece of paper and fill out the three variables below.

dV1 = [..., ..., ...] # Hill frame
dV2 = [..., ..., ...] # Hill frame
ToF = ...

# This function plots your Hohmann transfer trajectory, starting from an
# equatorial orbit at `a` = 7,000 km.

plots.plot_hohmann(dV1, dV2, ToF)